# `deque` 学习簿

`deque`（double-ended queue，双端队列）是 Python 标准库 **collections** 模块提供的一个高性能容器。  
它支持在序列两端以 **O(1) 时间复杂度** 进行快速插入和删除，适用于队列、栈、缓存等多种场景。

相比 `list`，`deque` 在两端操作时效率更高，非常适合实现 **FIFO 队列** 或 **LIFO 栈**。





## 准备工作

首先，我们需要从 `collections` 模块中导入 `deque`：


In [ ]:
from collections import deque
from time import perf_counter

## 创建与初始化

我们可以通过任何 **iterable** 来创建一个 `deque`：

In [ ]:
deque()

In [ ]:
deque((1, 2, 3, 4))

In [ ]:
deque([1, 2, 3, 4])

In [ ]:
deque(range(1, 5))

In [ ]:
deque("abcd")

In [ ]:
numbers = {"one": 1, "two": 2, "three": 3, "four": 4}
deque(numbers.keys())

In [ ]:
deque(numbers.values())

In [ ]:
deque(numbers.items())

## 基本操作

| 方法                 | 说明                      |
| ------------------ | ----------------------- |
| `append(x)`        | 在右端添加元素                 |
| `appendleft(x)`    | 在左端添加元素                 |
| `pop()`            | 移除并返回最右端元素              |
| `popleft()`        | 移除并返回最左端元素              |
| `extend(iter)`     | 在右端扩展多个元素               |
| `extendleft(iter)` | 在左端扩展多个元素（注意顺序会反转）      |
| `rotate(n)`        | 将队列旋转 n 步（正数向右，负数向左）    |
| `clear()`          | 清空队列                    |
| `maxlen`           | 创建时可设定最大长度，超出会自动删除另一端元素 |


deque 提供在两端进行插入（append）和弹出（pop）操作的极高性能。

### 删除元素（左侧）
使用 `popleft()` 方法可以移除并返回 **左端** 的元素。

In [ ]:
numbers = deque([1, 2, 3, 4])
numbers.popleft()

如果再执行一次popleft()，还会再一次移除并返回处理后的numbers左端的元素

In [ ]:
numbers.popleft()

In [ ]:
numbers

### 添加元素（左侧）
使用 `appendleft()` 方法可以在队列的 **左端** 添加元素。

In [ ]:
numbers.appendleft(2)
numbers

同样，如果再执行一次appendleft()，就会再一次添加并返回处理后的numbers左端的元素

In [ ]:
numbers.appendleft(1)
numbers

### 删除元素（右侧）
使用 `pop()` 方法可以移除并返回 **右端** 的元素。

In [ ]:
numbers.pop()

In [ ]:
numbers

## 性能对比

与 Python 内置的 `list` 相比，`deque` 在以下方面更优：
* `list.append()` 和 `list.pop()` 在尾部操作快，但在头部操作（`insert(0, x)`、`pop(0)`）很慢，因为涉及大量数据移动。
* `deque` 的两端操作时间复杂度都接近 **O(1)**。

我们可以使用 `time.perf_counter()` 来比较 `deque` 与 `list` 在两端操作时的性能差异，可以来看看下面的测试案例。

In [ ]:
TIMES = 10_000
a_list = []
a_deque = deque()

def average_time(func, times):
    total = 0.0
    for i in range(times):
        start = perf_counter()
        func(i)
        total += (perf_counter() - start) * 1e9
    return total / times

list_time = average_time(lambda i: a_list.insert(0, i), TIMES)
deque_time = average_time(lambda i: a_deque.appendleft(i), TIMES)
gain = list_time / deque_time

print(f"list.insert()      {list_time:.6} ns")
print(f"deque.appendleft() {deque_time:.6} ns  ({gain:.6}x faster)")

deque 完全支持 iterable 接口，也支持大部分 list 的方法，当然，切片（slice）不行。

In [ ]:
letters = deque("abde")

letters.insert(2, "c")
letters

In [ ]:
letters.remove("d")
letters

In [ ]:
letters[1]

In [ ]:
del letters[2]
letters

支持归支持，毕竟不是自己擅长的，所以性能就稍差些，但也没差太多，我们可以通过下面的测试来确认这一点。

In [ ]:
TIMES = 10_000
a_list = [1] * TIMES
a_deque = deque(a_list)

def average_time(func, times):
    total = 0.0
    for _ in range(times):
        start = perf_counter()
        func()
        total += (perf_counter() - start) * 1e6
    return total / times

def time_it(sequence):
    middle = len(sequence) // 2
    sequence.insert(middle, "middle")
    sequence[middle]
    sequence.remove("middle")
    del sequence[middle]

list_time = average_time(lambda: time_it(a_list), TIMES)
deque_time = average_time(lambda: time_it(a_deque), TIMES)
gain = deque_time / list_time

print(f"list  {list_time:.6} μs ({gain:.6}x faster)")
print(f"deque {deque_time:.6} μs")

对 deque 的一般操作，我们可以总结如下的性能对比：
| **操作** | **deque** | **list** |
| --- | --- | --- |
| 通过下标访问元素 | $O(n)$ | $O(1)$ |
| 左侧增加或弹出元素 | $O(1)$ | $O(n)$ |
| 右侧增加或弹出元素 | $O(1)$ | $O(1)+重分配内存时间$ |
| 中间插入或删除元素 | $O(n)$ | $O(n)$ |

下面我们来看看怎么把 deque 当队列（queue）用。我们来模拟一个顾客排队等待接待的场景。

In [ ]:
customers = deque()

customers.append("Jane")
customers.append("John")
customers.append("Linda")
customers

In [ ]:
customers.popleft()

In [ ]:
customers.popleft()

In [ ]:
customers.popleft()

In [ ]:
# 如果在一个空的 deque 上调用 popleft() 方法，会抛出 IndexError 异常
# customers.popleft()

如果需要我们可以简单地把 deque 包装一个标准的队列（queue）类。

In [ ]:
class Queue:
    def __init__(self):
        self._items = deque()

    def enqueue(self, item):
        self._items.append(item)

    def dequeue(self):
        try:
            return self._items.popleft()
        except IndexError:
            raise IndexError("dequeue from an empty queue") from None

    def __len__(self):
        return len(self._items)

    def __contains__(self, item):
        return item in self._items

    def __iter__(self):
        yield from self._items

    def __reversed__(self):
        yield from reversed(self._items)

    def __repr__(self):
        return f"Queue({list(self._items)})"

最后简介下 deque 的其他方法和用法。先来看 maxlen 属性，它可以限制 deque 的长度，当 deque 的长度超过 maxlen 时，会自动弹出左侧元素。

### 最大长度限制
创建 `deque` 时可以指定 `maxlen`，超过长度时会自动丢弃另一端的元素。

In [ ]:
four_numbers = deque([0, 1, 2, 3, 4], maxlen=4)
four_numbers

In [ ]:
four_numbers.append(5)
four_numbers

In [ ]:
four_numbers.append(6)  # Automatically remove 2
four_numbers

In [ ]:
four_numbers.appendleft(2) # Automatically remove 6
four_numbers

In [ ]:
four_numbers.appendleft(1)  # Automatically remove 5
four_numbers

In [ ]:
four_numbers.maxlen

另一个有用的方法是 rotate，就是把 deque 看做一个收尾相连，而 rotate 就是把里面的元素向右或者向左移动若干位置。

### 旋转操作
使用 `rotate(n)` 可以将队列中的元素整体右移 `n` 步（`n` 为负时左移）。

In [ ]:
ordinals = deque(["first", "second", "third", "fourth"])
ordinals.rotate()
ordinals

In [ ]:
ordinals.rotate(2)
ordinals

In [ ]:
ordinals.rotate(-1)
ordinals

还可以用 extend 和 extendleft 方法来一次往 deque 里添加一组元素，要特别注意 extendleft 是逆序扩展。

## 4. 高级操作

### 扩展元素（右侧）
使用 `extend()` 方法可以在 **右端** 一次性添加多个元素。

In [ ]:
numbers = deque([1, 2])
numbers.extend([3, 4, 5])
numbers

### 扩展元素（左侧）
使用 `extendleft()` 方法可以在 **左端** 一次性添加多个元素。
👉 注意：元素顺序会被反转。

In [ ]:
numbers.extendleft([-1, -2, -3, -4, -5])
numbers

deque 还支持 clear、copy、count、reverse、index 等方法，这些方法和 list 那边的用法基本一致。

In [ ]:
numbers = deque([1, 2, 2, 3, 4, 4, 5])

numbers + deque([6, 7, 8])

In [ ]:
numbers * 2

In [ ]:
numbers.index(2)

In [ ]:
numbers.count(4)

In [ ]:
numbers.reverse()
numbers

In [ ]:
numbers.clear()
numbers


## 7. 小结与练习

- `deque` 是一个双端队列，支持两端的快速插入和删除。
- 常见方法：`append`、`appendleft`、`pop`、`popleft`、`extend`、`rotate`、`maxlen`。
- 相比 `list`，在队首操作上 `deque` 更高效。

### 思考题
1. 使用 `deque` 实现一个队列（先进先出）。
2. 使用 `deque` 实现一个栈（后进先出）。
3. 尝试用 `rotate` 实现循环队列。
